In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tttttt/train_folds.csv
/kaggle/input/tttttt/train_dataset.csv
/kaggle/input/tttttt/test_dataset.csv


In [2]:
pip install wtfml

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import torch
import albumentations
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F
from wtfml.utils import EarlyStopping
from wtfml.engine import Engine

In [4]:
import torch
from torch.utils.data import Dataset
import numpy as np
import cv2
import albumentations as A

class CustomImageDataset(Dataset):
    def __init__(self, pixel_arrays, targets, resize=None, augmentations=None):
        self.pixel_arrays = pixel_arrays
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations

    def __len__(self):
        return len(self.pixel_arrays)

    def __getitem__(self, idx):
        # Convert the 1D array to a 2D grayscale image
        image = np.array(self.pixel_arrays[idx], dtype=np.float32).reshape(48, 48)
        
        # Resize image if needed
        if self.resize:
            image = cv2.resize(image, self.resize)

        # Expand dimensions to (48, 48, 1) and then convert to (new_size, new_size, 3) for RGB
        image = np.expand_dims(image, axis=-1)
        image = np.repeat(image, 3, axis=-1)  # Convert to RGB

        # Apply augmentations if specified
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']

        # Convert image to PyTorch tensor and permute to (3, new_size, new_size) format
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # For RGB, shape becomes (3, new_size, new_size)
        target = torch.tensor(self.targets[idx], dtype=torch.long)

        return image, target
import torch
import numpy as np
from sklearn import metrics
from tqdm import tqdm

class Engine:
    @staticmethod
    def train(data_loader, model, optimizer, device, scheduler=None, accumulation_steps=1, fp16=False):
        model.train()
        losses = AverageMeter()
        scaler = torch.cuda.amp.GradScaler() if fp16 else None
        
        if accumulation_steps > 1:
            optimizer.zero_grad()
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for batch_idx, (images, targets) in enumerate(tk0):
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                if fp16:
                    with torch.cuda.amp.autocast():
                        outputs = model(images)
                        loss = torch.nn.CrossEntropyLoss()(outputs, targets)
                    scaler.scale(loss).backward()
                else:
                    outputs = model(images)
                    loss = torch.nn.CrossEntropyLoss()(outputs, targets)
                    loss.backward()

                if (batch_idx + 1) % accumulation_steps == 0:
                    scaler.step(optimizer) if fp16 else optimizer.step()
                    if scheduler:
                        scheduler.step()
                    optimizer.zero_grad()

            losses.update(loss.item(), data_loader.batch_size)
            tk0.set_postfix(loss=losses.avg)
        
        return losses.avg

    @staticmethod
    def evaluate(data_loader, model, device, use_tpu=False):
        losses = AverageMeter()
        final_predictions = []
        model.eval()
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                images, targets = data  # Adjust if your data format is different

                # Move tensors to device
                images = images.to(device)
                targets = targets.to(device)

                # Forward pass
                predictions, loss = model(images, targets)
                predictions = predictions.cpu()
                losses.update(loss.item(), images.size(0))
                final_predictions.append(predictions)
                tk0.set_postfix(loss=losses.avg)

        # Concatenate all predictions and convert to NumPy array
        final_predictions = torch.cat(final_predictions).numpy()
        return final_predictions, losses.avg
    def predict(data_loader, model, device, use_tpu=False):
        model.eval()
        final_predictions = []
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                inputs, _ = data  # Unpack data
                inputs = inputs.to(device)
                predictions = model(inputs) 
                predictions = F.softmax(predictions,dim = 1)# Assume model returns only predictions
                final_predictions.append(predictions.cpu())
                tk0.set_postfix()
        return torch.cat(final_predictions).numpy()
        # Concatenate all predictions and convert to numpy array
        return torch.cat(final_predictions).numpy()
class AverageMeter():
        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count

In [5]:
import albumentations as A
from torch.utils.data import DataLoader
def train(fold):
    df = pd.read_csv("/kaggle/input/tttttt/train_folds.csv")
    device = "cuda"
    epochs = 50
    train_bs = 16
    valid_bs = 16

    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    train_pixels = df_train['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    train_targets = df_train['emotion'].values

    valid_pixels = df_valid['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    valid_targets = df_valid['emotion'].values

    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = A.Compose(
    [
        A.Resize(height=299, width=299),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
        A.HorizontalFlip(p=0.5)
    ]
    )

    valid_aug = A.Compose(
        [
        A.Resize(height=299, width=299),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True)
        ]
    )
    train_dataset = CustomImageDataset(
        pixel_arrays=train_pixels,
        targets=train_targets,
        resize=(299, 299),  # New size
        augmentations=train_aug,
    )

    train_loader = DataLoader(
        train_dataset, batch_size=16, shuffle=True, num_workers=4
    )

    valid_dataset = CustomImageDataset(
        pixel_arrays=valid_pixels,
        targets=valid_targets,
        resize=(299, 299),  # New size
        augmentations=valid_aug,
    )

    valid_loader = DataLoader(
        valid_dataset, batch_size=16, shuffle=False, num_workers=4
    )
    model = EF(num_classes= 7,pretrained='imagenet')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        threshold=0.001,
        mode="max"
    )

    es = EarlyStopping(patience=5, mode="max")
    for epoch in range(epochs):
        train_loss = Engine.train(train_loader, model,optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        fp16=False)
        predictions, valid_loss = Engine.evaluate(
            valid_loader, model, device=device
        )
        final_predictions = np.argmax(np.vstack(predictions), axis=1)
        valid_targets = np.array(valid_targets)  # Ensure targets are in the right format
    
    # Calculate accuracy
        accuracy = metrics.accuracy_score(valid_targets, final_predictions)
        print(f"Epoch = {epoch}, Accuracy = {accuracy:.4f}")
    
    # Update the learning rate scheduler
        scheduler.step(accuracy)
    
    # Early Stopping
        es(accuracy, model, model_path=f"model_fold_EFAD512{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break
    oof_data = {
        'id': df_valid.index,  # Use the index to map back to the original data
        'true_emotion': valid_targets,
        'pred_emotion': final_predictions,
        'fold': fold
    }
    
    return pd.DataFrame(oof_data)


In [6]:
pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=957252d960b347c1877941850cdc60c407dc66d5e70cb53e0b78d7319d9ff3fb
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from torch import nn
from efficientnet_pytorch import EfficientNet

import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

class EF(nn.Module):
    def __init__(self, num_classes=7, model_name='efficientnet-b6', pretrained='imagenet'):
        super(EF, self).__init__()
        
        # Load the pre-trained EfficientNet model
        self.base_model = EfficientNet.from_pretrained(model_name) if pretrained == 'imagenet' else EfficientNet.from_name(model_name)
        
        # Replace the final fully connected layer with the new layer for multi-class classification
        in_features = self.base_model._fc.in_features
        self.base_model._fc = nn.Linear(in_features, num_classes)
    
    def forward(self, image, targets=None):
        batch_size = image.size(0)
        
        # Forward pass through the base model
        out = self.base_model(image)
        
        # If targets are provided, calculate the loss
        if targets is not None:
            loss = nn.CrossEntropyLoss()(out, targets)
            return out, loss
        
        return out

In [8]:
dffold2 = train(1)
dffold3 = train(2)
dffold4 = train(3)
dffold5 = train(4)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth
100%|██████████| 165M/165M [00:01<00:00, 130MB/s]


Loaded pretrained weights for efficientnet-b6


100%|██████████| 63/63 [00:15<00:00,  3.98it/s, loss=1.32]


Epoch = 0, Accuracy = 0.4940
Validation score improved (-inf --> 0.494). Saving model!


100%|██████████| 63/63 [00:16<00:00,  3.92it/s, loss=1.17]


...

Epoch = 16, Accuracy = 0.5790
EarlyStopping counter: 3 out of 5


100%|██████████| 63/63 [00:16<00:00,  3.91it/s, loss=2.09]


Epoch = 17, Accuracy = 0.5630
EarlyStopping counter: 4 out of 5


100%|██████████| 63/63 [00:16<00:00,  3.91it/s, loss=2.02]


Epoch = 18, Accuracy = 0.5630
EarlyStopping counter: 5 out of 5
Early stopping


In [9]:
def predict(fold):
    df = pd.read_csv("/kaggle/input/emo-map-challenge/test_dataset.csv")
    device = "cuda"
    model_path=f"model_fold_EF(384){fold}.bin"
    test_pixels = df['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))                 
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    aug = albumentations.Compose(
        [
            albumentations.Resize(height=384, width=384),
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
        ]
    )
                                      
    targets = np.zeros(len(df))
    test_dataset = CustomImageDataset(
        pixel_arrays=test_pixels,
        targets=targets,
        resize=None,
        augmentations=aug,
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=4
    )

    model = model = EfficientNetForMultiClass(pretrained='imagenet')
    model.load_state_dict(torch.load(model_path))
    model.to(device)

    predictions = Engine.predict(test_loader, model, device=device)
    predictions = np.vstack((predictions))
    return predictions
